In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model

%matplotlib inline

In [2]:
infile_old = '../data/BEA-RegionalIncomeByIndustry/CA5_1969_2000_MSA.csv'
infile = '../data/BEA-RegionalIncomeByIndustry/CA5N_2001_2013_MSA.csv'

df_old = pd.read_csv(infile_old)
df = pd.read_csv(infile)

df_old


/usr/local/lib/python2.7/dist-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,GeoFIPS,GeoName,Region,Table,LineCode,IndustryClassification,Description,1969,1970,1971,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,10,...,Personal income,395971,443561,474234,...,2376433,2538036,2663665,2722081,2918059,3109606,3329032,3490073,3639311,3741391
1,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,20,...,Population (persons) 2/,125138,122505,124841,...,147400,149917,152909,153779,156097,156351,157405,158264,159755,160288
2,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,30,...,Per capita personal income (dollars),3164,3621,3799,...,16122,16930,17420,17701,18694,19889,21149,22052,22781,23342
3,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,35,...,Earnings by place of work,314729,345979,368356,...,1640392,1759145,1834506,1895221,1995628,2131048,2326369,2432863,2579609,2598895
4,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,36,...,less: Contributions for government social in...,18303,19934,22303,...,168262,176727,184213,193361,203789,214107,227052,232301,240171,244306
5,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,37,...,Employee and self-employed contributions f...,9557,10240,11245,...,87427,91460,95847,101582,108123,113405,120785,123957,128823,129272
6,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,38,...,Employer contributions for government soci...,8746,9694,11058,...,80835,85267,88366,91779,95666,100702,106267,108344,111348,115034
7,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,42,...,plus: Adjustment for residence 4/,1495,1863,2003,...,5066,6192,6245,4630,3561,3165,4426,6403,7632,10171
8,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,45,...,equals: Net earnings by place of residence,297921,327908,348056,...,1477196,1588610,1656538,1706490,1795400,1920106,2103743,2206965,2347070,2364760
9,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,46,...,"plus: Dividends, interest, and rent 5/",61986,73938,78915,...,542650,542405,579228,557362,624395,657686,668241,720342,709218,772092


In [3]:
#look at personal income per capita
manufacturing_df = df[df['LineCode']==500]
full_df = manufacturing_df.copy()

manufacturing_df_old = df_old[df_old['LineCode']==400]
full_df_old = manufacturing_df_old.copy()

manufacturing_df_old

,GeoFIPS,GeoName,Region,Table,LineCode,IndustryClassification,Description,1969,1970,1971,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
35,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,30607,31485,33877,...,168086,185028,172883,180286,196244,235405,259446,275008,281458,256299
148,10420,"Akron, OH (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,1060286,1055453,1064668,...,2694379,2814186,2931552,3066992,3181685,3210759,3261081,3452201,3525160,3550144
261,10500,"Albany, GA (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,55560,62238,70818,...,334435E,344168,367097,397446,403805,426007,445291,458972,446298,464784
374,10540,"Albany, OR (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,88042,93295,104284,...,387043,413382,425603,465909,484734,522609,571677,550060,531039,519179
487,10580,"Albany-Schenectady-Troy, NY (Metropolitan Stat...",NaN,CA5,400,D,Manufacturing,607201,635447,676356,...,1839149,1950834,1962203,1946153,1859729,1823878,1883775,1894562,1942989,2063389
600,10740,"Albuquerque, NM (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,73066,79484,89025,...,839612,869116,964481,1111113,1174515,1229029,1365433,(D),(D),(D)
713,10780,"Alexandria, LA (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,32472,33824,34014,...,147687,160447,175372,174055,178132,181415,178187,178220,199892,207283
826,10900,"Allentown-Bethlehem-Easton, PA-NJ (Metropolita...",NaN,CA5,400,D,Manufacturing,956379,996610,1003265,...,2851903,3097339,3161118,3248094,3343038,3210122,3359848,3379788,3540093,3675932
939,11020,"Altoona, PA (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,114165,117768,116559,...,290366,304083,324314,344225,351646,356851,373991,385914,417885,430935
1052,11100,"Amarillo, TX (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,61112E,67301E,75993E,...,353056E,370865E,398580E,431836E,458416E,476796,471859,(D),(D),(D)


In [4]:
# clean the data columns
import re

data_df = manufacturing_df[[str(x) for x in range(2001,2014)]]
data_df_old = manufacturing_df_old[[str(x) for x in range(1969,2001)]]

print data_df_old.head(3)

def clean_data_entry(x):
    m = re.match("\d+",x)
    if m:
        return float(m.group())
    else:
        return np.nan
        
data_df = data_df.applymap(clean_data_entry)
data_df_old = data_df_old.applymap(clean_data_entry)

print data_df_old.head(3)

        1969     1970     1971     1972     1973     1974     1975     1976  \
35     30607    31485    33877    41015    49523    62325    67428    66305   
148  1060286  1055453  1064668  1161319  1269333  1356121  1302663  1349946   
261    55560    62238    70818    86115   103444   114870   119231   133948   

        1977     1978   ...        1991     1992     1993     1994     1995  \
35     68129    74058   ...      168086   185028   172883   180286   196244   
148  1583276  1723508   ...     2694379  2814186  2931552  3066992  3181685   
261   154996   180613   ...     334435E   344168   367097   397446   403805   

        1996     1997     1998     1999     2000  
35    235405   259446   275008   281458   256299  
148  3210759  3261081  3452201  3525160  3550144  
261   426007   445291   458972   446298   464784  

[3 rows x 32 columns]
        1969     1970     1971     1972     1973     1974     1975     1976  \
35     30607    31485    33877    41015    49523    62325   

In [5]:
print len(data_df)
clean_df = data_df.dropna()
print len(clean_df)
inds = pd.isnull(data_df).any(1).nonzero()[0]

print len(inds)
print 'Cities with dropped inustries due to some withheld data:'
print manufacturing_df.iloc[inds]['GeoName'].unique()

print len(data_df_old)
clean_df_old = data_df_old.dropna()
print len(clean_df_old)
inds_old = pd.isnull(data_df_old).any(1).nonzero()[0]

print len(inds_old)
print 'Cities with dropped inustries due to some withheld data:'
print manufacturing_df_old.iloc[inds_old]['GeoName'].unique()

381
344
37
Cities with dropped inustries due to some withheld data:
['Albany, GA (Metropolitan Statistical Area)'
 'Albany-Schenectady-Troy, NY (Metropolitan Statistical Area)'
 'Amarillo, TX (Metropolitan Statistical Area)'
 'Augusta-Richmond County, GA-SC (Metropolitan Statistical Area)'
 'Billings, MT (Metropolitan Statistical Area)'
 'Boston-Cambridge-Newton, MA-NH (Metropolitan Statistical Area)'
 'Bowling Green, KY (Metropolitan Statistical Area)'
 'Charlottesville, VA (Metropolitan Statistical Area)'
 'Colorado Springs, CO (Metropolitan Statistical Area)'
 'Columbus, GA-AL (Metropolitan Statistical Area)'
 'Dothan, AL (Metropolitan Statistical Area)'
 'Dover, DE (Metropolitan Statistical Area)'
 'El Paso, TX (Metropolitan Statistical Area)'
 'Gulfport-Biloxi-Pascagoula, MS (Metropolitan Statistical Area)'
 'Hinesville, GA (Metropolitan Statistical Area)'
 'Jefferson City, MO (Metropolitan Statistical Area)'
 'Lafayette-West Lafayette, IN (Metropolitan Statistical Area)'
 'Lawton

In [6]:
relative_df = clean_df.copy()

means_df = clean_df.mean()

for year in relative_df.columns:
    relative_df[year] = data_df[year]/means_df[year]-1
    
print relative_df.head(3)

ind = relative_df.index

full_clean_df = pd.concat([full_df[full_df.columns[0:7]].loc[ind],relative_df],axis=1)

full_clean_df.head(3)


relative_df_old = clean_df_old.copy()

means_df_old = clean_df_old.mean()

for year in relative_df_old.columns:
    relative_df_old[year] = data_df_old[year]/means_df_old[year]-1
    
print relative_df_old.head(3)

ind_old = relative_df_old.index

full_clean_df_old = pd.concat([full_df_old[full_df_old.columns[0:7]].loc[ind_old],relative_df_old],axis=1)

full_clean_df_old.head(3)

         2001      2002      2003      2004      2005      2006      2007  \
34  -0.936605 -0.934891 -0.938062 -0.935296 -0.932210 -0.932420 -0.929749   
165  0.449273  0.468298  0.552881  0.583031  0.550055  0.538009  0.522961   
427 -0.750062 -0.752611 -0.745776 -0.742575 -0.731078 -0.721508 -0.728083   

         2008      2009      2010      2011      2012      2013  
34  -0.926248 -0.922652 -0.924509 -0.934732 -0.933980 -0.932813  
165  0.482843  0.437960  0.396082  0.441189  0.355539  0.341928  
427 -0.733027 -0.739858 -0.749537 -0.753761 -0.747733 -0.752332  
         1969      1970      1971      1972      1973      1974      1975  \
35  -0.927516 -0.925459 -0.920963 -0.912802 -0.906105 -0.890590 -0.883268   
148  1.510974  1.498797  1.483944  1.468956  1.406652  1.380636  1.255185   
261 -0.868423 -0.852651 -0.834777 -0.816920 -0.803870 -0.798349 -0.793586   

         1976      1977      1978    ...         1991      1992      1993  \
35  -0.897529 -0.908006 -0.911200    ... 

,GeoFIPS,GeoName,Region,Table,LineCode,IndustryClassification,Description,1969,1970,1971,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
35,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,-0.927516,-0.925459,-0.920963,...,-0.895866,-0.890304,-0.899773,-0.899406,-0.893433,-0.875508,-0.869790,-0.868090,-0.870485,-0.889271
148,10420,"Akron, OH (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,1.510974,1.498797,1.483944,...,0.669251,0.668417,0.699537,0.711289,0.727754,0.697985,0.636668,0.655882,0.622125,0.533764
261,10500,"Albany, GA (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,-0.868423,-0.852651,-0.834777,...,-0.792807,-0.795957,-0.787179,-0.778237,-0.780721,-0.774709,-0.776518,-0.779850,-0.794633,-0.799200


In [7]:
#consider one case to start modeling:

rand_example_df = full_clean_df[full_clean_df['GeoFIPS']==full_clean_df['GeoFIPS'].iloc[150]]
rand_example_df

rand_example_df_old = full_clean_df_old[full_clean_df_old['GeoFIPS']==full_clean_df['GeoFIPS'].iloc[150]]
rand_example_df_old

,GeoFIPS,GeoName,Region,Table,LineCode,IndustryClassification,Description,1969,1970,1971,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
18680,27100,"Jackson, MI (Metropolitan Statistical Area)",NaN,CA5,400,D,Manufacturing,-0.492484,-0.524343,-0.571317,...,-0.717664,-0.712093,-0.702657,-0.687609,-0.685432,-0.69073,-0.705012,-0.719011,-0.697317,-0.714765


In [12]:
data = rand_example_df[[str(x) for x in range(2001,2014)]]
years = data.columns.values
print type(years)
print (years)
values = data.values[0]
print type(values)
print values

data_old = rand_example_df_old[[str(x) for x in range(1969,2001)]]
years_old = data_old.columns.values
print (years_old)
values_old = data_old.values[0]
print values_old

years_comb = np.concatenate((years_old,years),axis=0)
print len(years_comb),years_comb
values_comb = np.concatenate((values_old,values),axis=0)
print len(values_comb),values_comb

<type 'numpy.ndarray'>
['2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009' '2010'
 '2011' '2012' '2013']
<type 'numpy.ndarray'>
[-0.706894   -0.6954318  -0.68807254 -0.69183617 -0.68735352 -0.69645245
 -0.70896564 -0.71342532 -0.73376221 -0.72739083 -0.71402293 -0.70875649
 -0.69514305]
['1969' '1970' '1971' '1972' '1973' '1974' '1975' '1976' '1977' '1978'
 '1979' '1980' '1981' '1982' '1983' '1984' '1985' '1986' '1987' '1988'
 '1989' '1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998'
 '1999' '2000']
[-0.49248428 -0.52434336 -0.57131721 -0.53648898 -0.50728339 -0.53935988
 -0.55570708 -0.59856131 -0.59325965 -0.58695398 -0.59894633 -0.63912384
 -0.63964725 -0.678188   -0.70151008 -0.71166347 -0.7087333  -0.70501427
 -0.69471964 -0.68965032 -0.67705588 -0.69694966 -0.71766431 -0.71209264
 -0.70265673 -0.68760897 -0.6854319  -0.69073039 -0.70501167 -0.7190114
 -0.69731701 -0.7147652 ]
45 ['1969' '1970' '1971' '1972' '1973' '1974' '1975' '1976' '1977' '1978'
 '1979'

In [14]:
def windowedSVR(X, Y, ntest=6, window=5, kernel='linear'):
    def getWindows(arr):
        numPts = len(arr)
        print numPts
        
        i=0
        j=window
        x = []
        y = []
        while j < numPts:
            x.append(arr[i:j])
            i += 1
            j += 1

        j = window
        while j < numPts:
            y.append(arr[j])
            j += 1
        return x, y
    
    trans = StandardScaler()
    Y = trans.fit_transform(Y)
    
    Ytrain = Y[0:-ntest]
    Ytest = Y[-ntest:]
    
    Xtrain = X[0:-ntest]
    Xtest = X[-ntest:]

    xtrain, ytrain = getWindows(Ytrain)
    
    Ytrain = trans.inverse_transform(Ytrain)
    Ytest = trans.inverse_transform(Ytest)
    
    print 'Ytrain',len(Ytrain),Ytrain
    print 'ytrain',len(ytrain),ytrain
    print 'Ytest',len(Ytest),Ytest
    
    #windowed svr gridsearch
    if(kernel=='rbf'):
        parameters = {'gamma':[2**i for i in range(-30,30)], 'C':[2**i for i in range(-30,30)]}
    if(kernel=='linear'):
        parameters = {'C':[2**i for i in range(-30,30)]}
    svr = SVR(kernel=kernel)

    r2_scorer = make_scorer(r2_score)
    
    grid = GridSearchCV(svr, param_grid=parameters)
    fitted_svr = grid.fit(xtrain, ytrain).best_estimator_
    ypred = fitted_svr.predict(xtest)
    
    score = r2_scorer(ytest,ypred)

    ytrain = trans.inverse_transform(ytrain)
    ytest = trans.inverse_transform(ytest)
    ypred = trans.inverse_transform(ypred)

    print(score)
    print(len(ypred))

    plt.plot_date(Xtrain, Ytrain, '--', color='green')
    plt.plot_date(Xtest[window:], ypred, '-o', color='blue')
    plt.plot_date(Xtest, Ytest, '-x', color='red')

In [ ]:
windowedSVR(years_comb, values_comb)

In [11]:
#prepare windowed data:
window = 5
datapoints = len(values_comb)

trans = StandardScaler()
values_comb = trans.fit_transform(values_comb)

a = 0
b = window
x = []
y = []
while b < datapoints:
    x.append(values_comb[a:b])
    a += 1
    b += 1

b = window
while b < datapoints:
    y.append(values_comb[b])
    b += 1
    

values_comb = trans.inverse_transform(values_comb)
    
print len(x), x
print len(y), y

40 [array([ 2.57406654,  2.09386487,  1.38584329,  1.91079794,  2.35100434]), array([ 2.09386487,  1.38584329,  1.91079794,  2.35100434,  1.86752579]), array([ 1.38584329,  1.91079794,  2.35100434,  1.86752579,  1.62112972]), array([ 1.91079794,  2.35100434,  1.86752579,  1.62112972,  0.97520181]), array([ 2.35100434,  1.86752579,  1.62112972,  0.97520181,  1.05511205]), array([ 1.86752579,  1.62112972,  0.97520181,  1.05511205,  1.15015544]), array([ 1.62112972,  0.97520181,  1.05511205,  1.15015544,  0.96939855]), array([ 0.97520181,  1.05511205,  1.15015544,  0.96939855,  0.36381599]), array([ 1.05511205,  1.15015544,  0.96939855,  0.36381599,  0.35592679]), array([ 1.15015544,  0.96939855,  0.36381599,  0.35592679, -0.22498543]), array([ 0.96939855,  0.36381599,  0.35592679, -0.22498543, -0.57651143]), array([ 0.36381599,  0.35592679, -0.22498543, -0.57651143, -0.72955022]), array([ 0.35592679, -0.22498543, -0.57651143, -0.72955022, -0.68538477]), array([-0.22498543, -0.57651143, -

In [9]:
###### windowed svr gridsearch  THIS ONE WORKS
save_for_test = window +1



#parameters = {'gamma':[2**i for i in range(-30,30)], 'C':[2**i for i in range(-30,30)]}
#svr = SVR(kernel='rbf')
parameters = {'C':[2**i for i in range(-30,30)]}
svr = SVR(kernel='linear')

r2_scorer = make_scorer(r2_score)
grid = GridSearchCV(svr, param_grid=parameters)
svr_grid = grid.fit(x[:-save_for_test], y[:-save_for_test]).best_estimator_
print svr_grid
print grid
y_rbf = svr_grid.predict(x)
score = svr_grid.score(x[-7:],y[-7:])

y = trans.inverse_transform(y)
y_rbf = trans.inverse_transform(y_rbf)

print score
print len(y_rbf)
print y_rbf
print y

plt.plot(years_comb, values_comb, '-o', color='green')
plt.plot(years_comb[window:], y_rbf, '-o', color='blue')

#plt.ylim(-.5,.5)
plt.show()



NameError: name 'window' is not defined

In [ ]:
linear_kern_window_size_r2s = {}

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()
las_reg = linear_model.Lasso(alpha = .5)

# Train the model using the training sets
values_pred = las_reg.fit(np.matrix(years).T, values).predict(np.matrix(years).T)

score = las_reg.score(np.matrix(years).T, values)

print values_pred
print values
print score

plt.plot(years, values, '-o', color='green')
plt.plot(years, values_pred, '-o', color='blue')

plt.ylim(-.8,0)
plt.show()

In [ ]:
print values

trans = StandardScaler()
trans_values = trans.fit_transform(values)

trans.inverse_transform(trans_values)